In [26]:
import pandas as pd
import json
import requests

In [27]:
df_production   = pd.read_csv( '../../data/production.csv', low_memory=False )
df_store        = pd.read_csv( '../../data/store.csv', low_memory=False )

# merge
df_test = pd.merge( df_production, df_store, how='left', on='Store' )

In [ ]:
# Escolha de lojas para entrarem na previsão
df_test = df_test[df_test['Store'].isin( [20, 30, 32] )]

# Remover dias de lojas fechadas
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]

In [ ]:
# Converter Dataframe para json
data = json.dumps( df_test.to_dict( orient='records' ) )

In [30]:
# API Call
url = 'http://10.5.0.2:5000/rossmann/predict'
header = {'Content-type': 'application/json' } 
data = data

r = requests.post( url, data=data, headers=header )
print( 'Status Code {}'.format( r.status_code ) )

Status Code 200


In [31]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys() )

In [32]:
d2 = d1[['Store', 'predictions']].groupby( 'Store' ).sum().reset_index()

for i in range( len( d2 ) ):
    print( 'Store Number {} will sell R${:,.2f} in the next 6 weeks'.format( 
            d2.loc[i, 'Store'], 
            d2.loc[i, 'predictions'] ) )

Store Number 20 will sell R$277,778.36 in the next 6 weeks
Store Number 30 will sell R$213,744.31 in the next 6 weeks
Store Number 32 will sell R$152,042.77 in the next 6 weeks
